In [1]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F

In [14]:
class BiLinUnit(torch.nn.Module):
    def __init__(self, dimIn, dim, dW, dW2, dropout=0.):
        super(BiLinUnit, self).__init__()
        self.conv1 = torch.nn.Conv2d(dimIn, 2 * dim, (2 * dW + 1, 2 * dW + 1), padding=dW, bias=False)
        self.conv2 = torch.nn.Conv2d(2 * dim, dim, (2 * dW2 + 1, 2 * dW2 + 1), padding=dW2, bias=False)
        self.conv3 = torch.nn.Conv2d(2 * dim, dimIn, (2 * dW2 + 1, 2 * dW2 + 1), padding=dW2, bias=False)
        self.bilin0 = torch.nn.Conv2d(dim, dim, (2 * dW2 + 1, 2 * dW2 + 1), padding=dW2, bias=False)
        self.bilin1 = torch.nn.Conv2d(dim, dim, (2 * dW2 + 1, 2 * dW2 + 1), padding=dW2, bias=False)
        self.bilin2 = torch.nn.Conv2d(dim, dim, (2 * dW2 + 1, 2 * dW2 + 1), padding=dW2, bias=False)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, xin):
        x = self.conv1(xin)
        x = self.dropout(x)
        x = self.conv2(F.relu(x))
        x = self.dropout(x)
        x = torch.cat((self.bilin0(x), self.bilin1(x) * self.bilin2(x)), dim=1)
        x = self.dropout(x)
        x = self.conv3(x)
        return x


class Encoder(torch.nn.Module):
    def __init__(self, dimInp, dimAE, dW, dW2, sS, nbBlocks, rateDropout=0.):
        super(Encoder, self).__init__()

        self.NbBlocks = nbBlocks
        self.DimAE = dimAE
        # self.conv1HR  = torch.nn.Conv2d(dimInp,self.DimAE,(2*dW+1,2*dW+1),padding=dW,bias=False)
        # self.conv1LR  = torch.nn.Conv2d(dimInp,self.DimAE,(2*dW+1,2*dW+1),padding=dW,bias=False)
        #self.pool1 = torch.nn.AvgPool2d(sS)
        self.pool1 = torch.nn.AvgPool2d((1,sS))
        #self.convTr = torch.nn.ConvTranspose2d(dimInp, dimInp, (sS, sS), stride=(sS, sS), bias=False)
        self.convTr = torch.nn.ConvTranspose2d(dimInp, dimInp, (1, sS), stride=(1, sS), bias=False)
        
        # self.NNtLR    = self.__make_ResNet(self.DimAE,self.NbBlocks,rateDropout)
        # self.NNHR     = self.__make_ResNet(self.DimAE,self.NbBlocks,rateDropout)
        self.NNLR = self.__make_BilinNN(dimInp, self.DimAE, dW, dW2, self.NbBlocks, rateDropout)
        self.NNHR = self.__make_BilinNN(dimInp, self.DimAE, dW, dW2, self.NbBlocks, rateDropout)
        self.dropout = torch.nn.Dropout(rateDropout)

    def __make_BilinNN(self, dimInp, dimAE, dW, dW2, Nb_Blocks=2, dropout=0.):
        layers = []
        layers.append(BiLinUnit(dimInp, dimAE, dW, dW2, dropout))
        for kk in range(0, Nb_Blocks - 1):
            layers.append(BiLinUnit(dimAE, dimAE, dW, dW2, dropout))
        return torch.nn.Sequential(*layers)

    def forward(self, xinp):
        ## LR component
        xLR = self.NNLR(self.pool1(xinp))
        xLR = self.dropout(xLR)
        xLR = self.convTr(xLR)
        # HR component
        xHR = self.NNHR(xinp)
        return xLR + xHR


class Decoder(torch.nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

    def forward(self, x):
        return torch.mul(1., x)


class CorrelateNoise(torch.nn.Module):
    def __init__(self, shape_data, dim_cn):
        super(CorrelateNoise, self).__init__()
        self.conv1 = torch.nn.Conv2d(shape_data, dim_cn, (3, 3), padding=1, bias=False)
        self.conv2 = torch.nn.Conv2d(dim_cn, 2 * dim_cn, (3, 3), padding=1, bias=False)
        self.conv3 = torch.nn.Conv2d(2 * dim_cn, shape_data, (3, 3), padding=1, bias=False)

    def forward(self, w):
        w = self.conv1(F.relu(w)).to(device)
        w = self.conv2(F.relu(w)).to(device)
        w = self.conv3(w).to(device)
        return w


class RegularizeVariance(torch.nn.Module):
    def __init__(self, shape_data, dim_rv):
        super(RegularizeVariance, self).__init__()
        self.conv1 = torch.nn.Conv2d(shape_data, dim_rv, (3, 3), padding=1, bias=False)
        self.conv2 = torch.nn.Conv2d(dim_rv, 2 * dim_rv, (3, 3), padding=1, bias=False)
        self.conv3 = torch.nn.Conv2d(2 * dim_rv, shape_data, (3, 3), padding=1, bias=False)

    def forward(self, v):
        v = self.conv1(F.relu(v)).to(device)
        v = self.conv2(F.relu(v)).to(device)
        v = self.conv3(v).to(device)
        return v

class Phi_r(torch.nn.Module):
    def __init__(self, shapeData, DimAE, dW, dW2, sS, nbBlocks, rateDr, stochastic=False):
        super(Phi_r, self).__init__()
        self.encoder = Encoder(shapeData, DimAE, dW, dW2, sS, nbBlocks, rateDr)
        self.decoder = Decoder()
        self.correlate_noise = CorrelateNoise(shapeData, 10)
        self.regularize_variance = RegularizeVariance(shapeData, 10)
        self.stochastic = stochastic

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        if self.stochastic == True:
            W = torch.randn(x.shape).to(device)
            #  g(W) = alpha(x)*h(W)
            gW = torch.mul(self.regularize_variance(x),self.correlate_noise(W))
            # variance
            gW = gW/torch.std(gW)
            # print(stats.describe(gW.detach().cpu().numpy()))
            x = x + gW
        return x


In [17]:
Phi_r(1,2,3,4,5,6,0)

Phi_r(
  (encoder): Encoder(
    (pool1): AvgPool2d(kernel_size=(1, 5), stride=(1, 5), padding=0)
    (convTr): ConvTranspose2d(1, 1, kernel_size=(1, 5), stride=(1, 5), bias=False)
    (NNLR): Sequential(
      (0): BiLinUnit(
        (conv1): Conv2d(1, 4, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
        (conv2): Conv2d(4, 2, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4), bias=False)
        (conv3): Conv2d(4, 1, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4), bias=False)
        (bilin0): Conv2d(2, 2, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4), bias=False)
        (bilin1): Conv2d(2, 2, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4), bias=False)
        (bilin2): Conv2d(2, 2, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4), bias=False)
        (dropout): Dropout(p=0, inplace=False)
      )
      (1): BiLinUnit(
        (conv1): Conv2d(2, 4, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
        (conv2): Conv2d(4, 2, kernel_size=

In [2]:

class CovarianceNetwork(torch.nn.Module):
    def __init__(self, shapeData, dimout, dimH):
        super(CovarianceNetwork,self).__init__()
        self.layer1 = torch.nn.Linear(shapeData, dimH)
        self.layer2 = torch.nn.Linear(dimH, dimout)

    def forward(self, w):
        w = self.layer1(F.relu(w))
        w = self.layer2(F.relu(w))
        w = torch(w*w)
        return w

In [3]:
C=CovarianceNetwork(2,6,4)
print(C.layer1.weight)
x = torch.rand(20,4,2)
print(x.shape)
print(C(x))
print(C(x).shape)
print(device(x))

SyntaxError: cannot assign to function call (<ipython-input-3-876e573f8aac>, line 1)

In [10]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon May 18 17:59:23 2020
@author: rfablet
"""

#######################################

import argparse
import numpy as np
import matplotlib.pyplot as plt 
#import os
#import tensorflow.keras as keras
import xarray as xr

import time
import copy
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import scipy
from scipy.integrate import solve_ivp
from sklearn.feature_extraction import image
from netCDF4 import Dataset
import datetime

# specific torch module 
#import dinAE_solver_torch as dinAE
#import torch4DVarNN_solver as NN_4DVar
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from netCDF4 import Dataset
from statsmodels.distributions.empirical_distribution import ECDF



############################

parser = argparse.ArgumentParser()
flagProcess    = [0,1,2,3,4]#Sequence fo processes to be run
    
flagRandomSeed = 0
flagSaveModel  = 1
     
batch_size  = 256#4#4#8#12#8#256#8 originellement 96

dirSAVE     = './ResDanube4DVar/'
suffix_exp='exp3'
genFilename = 'Debit_v11'
  
flagAEType = 2 # 0: L96 model, 1-2: GENN
DimAE      = 50#50#10#50
    
UsePriodicBoundary = True # use a periodic boundary for all conv operators in the gradient model (see torch_4DVarNN_dinAE)
InterpFlag         = False

NbDays          = 18244

time_step  = 1
DT = 13
sigNoise   = np.sqrt(2)
rateMissingData = 0.5#0.9
width_med_filt_spatial = 5
width_med_filt_temp = 1


# loss weghing wrt time
w_ = np.zeros(DT)
w_[int(DT / 2)] = 1.
wLoss = torch.Tensor(w_)

flagTypeMissData = 4
print('........ Data extraction')
if flagRandomSeed == 0:
    print('........ Random seed set to 100')
    np.random.seed(100)
        
###############################################################
## data extraction
ncfile = Dataset('Dataset_danube_Copy1.nc',"r")
L=[]
for i in range(31):
    L.append(ncfile['S'+str(i+1)][:].reshape(18244,1))
        
dataset = np.concatenate((L[0],L[1],L[2],L[3],L[4],L[5],L[6],L[7],L[8],L[9],L[10],L[11],L[12],L[13],L[14],L[15],L[16],L[17],L[18],L[19],L[20],L[21],L[22],L[23],L[24],L[25],L[26],L[27],L[28],L[29],L[30]),axis=1)

seuil_10 = np.zeros(31)
#for i in range(31) :
#    Si = sorted(L[i],reverse = True)[int(18244/10)]
#    print(Si)
#    seuil_10[i]=Si[0]
seuil_10 =np.array([2220.,  1020. ,  764.,   744. ,  577. ,  535.,   511.,   329.,   279. ,  218.,
   92.3 ,  45.,  1240.,   257. ,  250.,   116. ,   81.5,   46.7,   35.3,  179.,
  130.,   123.,    96.5,   78.6 ,  69.,    62.1,   47.6 ,  81.8,   74.4 , 449.,
  427. ])    
print(seuil_10)


# Definiton of training, validation and test dataset    
i=0
Indtrain=[]
Indval=[]
Indtest=[]
while (i+1)*395<(NbDays-1):
    x=395*i
    Indtrain.append([x,(x+305)])
    Indval.append([(x+319),(x+350)])
    Indtest.append([x+364,x+395])
    i+=1
    

#Se restreindre à l'été car pas de pluie??
day0=datetime.date(1960,1,1)
dayend=datetime.date(2009,12,12)


#Trouver une valeur de seuil pour étudier le coût KL au-dessus du seuil : on se place dans flagTypeProcess 3 avec la station 4 masquée

D=dataset[Indtrain[0][0]:Indtrain[0][1],3]
for k in Indtrain[1::]:
    D=np.concatenate((D,dataset[k[0]:k[1],3]),axis=0)
r=0.1 #fraction supérieure 
D.sort()
seuil=D[int((1-r)*len(D))]



    
    ####################################################
## Generation of training  validationand test dataset
## Extraction of time series of dT time steps
#NbTraining = 6000#2000
#NbTest     = 256#256#500
#NbVal = ?
    
dataTrainingNoNaND = image.extract_patches_2d(dataset[Indtrain[0][0]:Indtrain[0][1],:],(DT,31)) 
for k in Indtrain[1::]:
    d= image.extract_patches_2d(dataset[k[0]:k[1],:],(DT,31))
    dataTrainingNoNaND=np.concatenate((dataTrainingNoNaND,d),axis=0)
        
    
dataValNoNaND = image.extract_patches_2d(dataset[Indval[0][0]:Indval[0][1],:],(DT,31))    
for k in Indval[1::]:
    d= image.extract_patches_2d(dataset[k[0]:k[1],:],(DT,31))
    dataValNoNaND=np.concatenate((dataValNoNaND,d),axis=0)
print(dataValNoNaND.shape )  
    
dataTestNoNaND = image.extract_patches_2d(dataset[Indtest[0][0]:Indtest[0][1],:],(DT,31))
for k in Indtest[1::]:
    d= image.extract_patches_2d(dataset[k[0]:k[1],:],(DT,31))
    dataTestNoNaND=np.concatenate((dataTestNoNaND,d),axis=0)
print(dataTestNoNaND.shape ) 
        
# create missing data
#flagTypeMissData = 0 : Missing data randomly chosen on the patch driven by rateMissingData
#flagTypeMissData = 1 : Almost the same
#flagTypeMissData = 2 : In each patch, different station are randomly chosen and are masked according to rateMissingData
#flagTypeMissData = 3 : The same stations listed in MaskedStations are masked
#flagTpeMissData = 4  : Prevision

if flagTypeMissData == 0:
    indRandD         = np.random.permutation(dataTrainingNoNaND.shape[0]*dataTrainingNoNaND.shape[1]*dataTrainingNoNaND.shape[2])
    indRandD         = indRandD[0:int(rateMissingData*len(indRandD))]
    dataTrainingD    = np.copy(dataTrainingNoNaND).reshape((dataTrainingNoNaND.shape[0]*dataTrainingNoNaND.shape[1]*dataTrainingNoNaND.shape[2],1))
    dataTrainingD[indRandD] = float('nan')
    dataTrainingD    = np.reshape(dataTrainingD,(dataTrainingNoNaND.shape[0],dataTrainingNoNaND.shape[1],dataTrainingNoNaND.shape[2]))
            
    indRandD         = np.random.permutation(dataValNoNaND.shape[0]*dataValNoNaND.shape[1]*dataValNoNaND.shape[2])
    indRandD         = indRandD[0:int(rateMissingData*len(indRandD))]
    dataValD    = np.copy(dataValNoNaND).reshape((dataValNoNaND.shape[0]*dataValNoNaND.shape[1]*dataValNoNaND.shape[2],1))
    dataValD[indRandD] = float('nan')
    dataValD    = np.reshape(dataValD,(dataValNoNaND.shape[0],dataValNoNaND.shape[1],dataValNoNaND.shape[2]))
            
            
    indRandD         = np.random.permutation(dataTestNoNaND.shape[0]*dataTestNoNaND.shape[1]*dataTestNoNaND.shape[2])
    indRandD         = indRandD[0:int(rateMissingData*len(indRandD))]
    dataTestD        = np.copy(dataTestNoNaND).reshape((dataTestNoNaND.shape[0]*dataTestNoNaND.shape[1]*dataTestNoNaND.shape[2],1))
    dataTestD[indRandD] = float('nan')
    dataTestD          = np.reshape(dataTestD,(dataTestNoNaND.shape[0],dataTestNoNaND.shape[1],dataTestNoNaND.shape[2]))

    genSuffixObs    = '_ObsRnd_%02d_%02d'%(100*rateMissingData,10*sigNoise**2)
        
elif flagTypeMissData==1:
    time_step_obs   = int(1./(1.-rateMissingData))
    dataTrainingD    = np.zeros((dataTrainingNoNaND.shape))
    dataTrainingD[:] = float('nan')
            
    dataValD    = np.zeros((dataValNoNaND.shape))
    dataValD[:] = float('nan')
            
    dataTestD        = np.zeros((dataTestNoNaND.shape))
    dataTestD[:]     = float('nan')
               
    if 1*0:
                
        dataTrainingD[:,::time_step_obs,:] = dataTrainingNoNaND[:,::time_step_obs,:]
        dataValD[:,::time_step_obs,:] = dataValNoNaND[:,::time_step_obs,:]
        dataTestD[:,::time_step_obs,:]     = dataTestNoNaND[:,::time_step_obs,:]
                    
        genSuffixObs    = '_ObsSub_%02d_%02d'%(100*rateMissingData,10*sigNoise**2)
    else:
        for nn in range(0,dataTrainingD.shape[1],time_step_obs):
            indrand = np.random.permutation(dataTrainingD.shape[2])[0:int(0.5*dataTrainingD.shape[1])]
            dataTrainingD[:,nn,indrand] = dataTrainingNoNaND[:,nn,indrand]
                    
        for nn in range(0,dataTrainingD.shape[1],time_step_obs):
            indrand = np.random.permutation(dataTrainingD.shape[2])[0:int(0.5*dataTrainingD.shape[1])]
            dataValD[:,nn,indrand] = dataValNoNaND[:,nn,indrand]
                    
        for nn in range(0,dataTrainingD.shape[1],time_step_obs):
            indrand = np.random.permutation(dataTrainingD.shape[2])[0:int(0.5*dataTrainingD.shape[1])]
            dataTestD[:,nn,indrand] = dataTestNoNaND[:,nn,indrand]

        genSuffixObs    = '_ObsSubRnd_%02d_%02d'%(100*rateMissingData,10*sigNoise**2)
        
elif flagTypeMissData == 2 :
    #
    Nbtraining=13110
    Nbval=506
    Nbtest=506
            
    ratemissingdata_space = 0.15
    time_step_obs   = int(1./(1.-rateMissingData))
    dataTrainingD    = np.zeros(([Nbtraining,dataTrainingNoNaND.shape[1],dataTrainingNoNaND.shape[2]]))
    dataTrainingD[:] = float('nan')
    dataTrainingNoNaND2    = np.zeros(([Nbtraining,dataTrainingNoNaND.shape[1],dataTrainingNoNaND.shape[2]]))
    dataTrainingNoNaND2[:] = float('nan')
            
    dataValD    = np.zeros(([Nbval,dataTrainingNoNaND.shape[1],dataTrainingNoNaND.shape[2]]))           
    dataValD[:] = float('nan')
    dataValNoNaND2=np.zeros(([Nbval,dataTrainingNoNaND.shape[1],dataTrainingNoNaND.shape[2]]))
    dataValNoNaND2[:] = float('nan')
            
    dataTestD        = np.zeros(([Nbtest,dataTrainingNoNaND.shape[1],dataTrainingNoNaND.shape[2]]))
    dataTestD[:]     = float('nan') 
    dataTestNoNaND2 =np.zeros(([Nbtest,dataTrainingNoNaND.shape[1],dataTrainingNoNaND.shape[2]]))
    dataTestNoNaND2[:] = float('nan')
            
    ind=0
    print(dataTrainingD.shape)
    
    while ind<Nbtraining:
        indrand=np.random.permutation(dataTrainingD.shape[2])[0:int((1-ratemissingdata_space)*dataTrainingD.shape[2])]
        dataTrainingD[ind,:,indrand]=dataTrainingNoNaND[ind%dataTrainingNoNaND.shape[0],:,indrand]
        dataTrainingNoNaND2[ind,:,:]=dataTrainingNoNaND[ind%dataTrainingNoNaND.shape[0],:,:]
                
        if ind <Nbval:
            indrand2=np.random.permutation(dataTrainingD.shape[2])[0:int((1-ratemissingdata_space)*dataTrainingD.shape[2])]
            dataValD[ind,:,indrand2]=dataValNoNaND[ind%dataValNoNaND.shape[0],:,indrand2]
            dataValNoNaND2[ind,:,:]=dataValNoNaND[ind%dataValNoNaND.shape[0],:,:]
                
            indrand3=np.random.permutation(dataTrainingD.shape[2])[0:int((1-ratemissingdata_space)*dataTrainingD.shape[2])]
            dataTestD[ind,:,indrand3]=dataTestNoNaND[ind%dataTestNoNaND.shape[0],:,indrand3]
            dataTestNoNaND2[ind,:,:] = dataTestNoNaND[ind%dataTestNoNaND.shape[0],:,:]
        ind+=1
                
    dataTrainingNoNaND =dataTrainingNoNaND2
    dataValNoNaND = dataValNoNaND2
    dataTestNoNaND =dataTestNoNaND2        
    genSuffixObs    = '_ObsSubRnd_%02d_%02d'%(100*rateMissingData,10*sigNoise**2) 
        
#mask only on specific station
elif flagTypeMissData == 3 :
    MaskedStations=[2,4,16,25]
            
    dataTrainingD    = np.zeros((dataTrainingNoNaND.shape))
    dataTrainingD[:] = float('nan')
            
    dataValD    = np.zeros((dataValNoNaND.shape))
    dataValD[:] = float('nan')
            
    dataTestD        = np.zeros((dataTestNoNaND.shape))
    dataTestD[:]     = float('nan')
    print(dataTrainingNoNaND[0,:,:])  
    for i in range(31):
        dataTrainingD[:,:,i] = dataTrainingNoNaND[:,:,i]
        dataValD[:,:,i] = dataValNoNaND[:,:,i]
        dataTestD[:,:,i] = dataTestNoNaND[:,:,i]
    for i in MaskedStations:
        dataTrainingD[:,:,i-1] = float('nan')
        dataValD[:,:,i-1] = float('nan')
        dataTestD[:,:,i-1] = float('nan')
    genSuffixObs    = '_ObsSubRnd_%02d_%02d'%(100*rateMissingData,10*sigNoise**2)
    print(dataTrainingNoNaND[0,:,:])
    
else : 
    dataTrainingD    = np.zeros((dataTrainingNoNaND.shape))
    dataTrainingD[:] = float('nan')
            
    dataValD    = np.zeros((dataValNoNaND.shape))
    dataValD[:] = float('nan')
            
    dataTestD        = np.zeros((dataTestNoNaND.shape))
    dataTestD[:]     = float('nan')
    
    dataTrainingD[:,:(int(2*DT/3+1)),:] = dataTrainingNoNaND[:,:(int(2*DT/3+1)),:]
    dataValD[:,:(int(2*DT/3+1)),:] = dataValNoNaND[:,:(int(2*DT/3+1)),:]
    dataTestD[:,:(int(2*DT/3+1)),:] = dataTestNoNaND[:,:(int(2*DT/3+1)),:]
    genSuffixObs    = '_ObsSubRnd_%02d_%02d'%(100*rateMissingData,10*sigNoise**2)
    
print('... Data type: '+genSuffixObs)
    #for nn in range(0,dataTraining.shape[1],time_step_obs):
    #    dataTraining[:,::time_step_obs,:] = dataTrainingNoNaN[:,::time_step_obs,:]
    #dataTest    = np.zeros((dataTestNoNaN.shape))
    #dataTest[:] = float('nan')
    #dataTest[:,::time_step_obs,:] = dataTestNoNaN[:,::time_step_obs,:]
        
# set to NaN patch boundaries    
if 1*0:
    dataTrainingD[:,0:10,:] =  float('nan')
    dataValD[:,0:10,:] =  float('nan')
    dataTestD[:,0:10,:]     =  float('nan')
    dataTrainingD[:,dT-10:dT,:] =  float('nan')
    dataValD[:,dT-10:dT,:] =  float('nan')
    dataTestD[:,dT-10:dT,:]     =  float('nan')
            
                                
# mask for NaN
maskTrainingD = (dataTrainingD == dataTrainingD).astype('float')
maskValD = (dataValD == dataValD).astype('float')
maskTestD     = ( dataTestD    ==  dataTestD   ).astype('float')
            
dataTrainingD = np.nan_to_num(dataTrainingD)
        
dataValD = np.nan_to_num(dataValD)
dataTestD     = np.nan_to_num(dataTestD)
            
    # Permutation to have channel as #1 component
dataTrainingD      = np.moveaxis(dataTrainingD,-1,1)
maskTrainingD      = np.moveaxis(maskTrainingD,-1,1)
dataTrainingNoNaND = np.moveaxis(dataTrainingNoNaND,-1,1)
        
dataValD      = np.moveaxis(dataValD,-1,1)
maskValD      = np.moveaxis(maskValD,-1,1)
dataValNoNaND = np.moveaxis(dataValNoNaND,-1,1)
            
dataTestD      = np.moveaxis(dataTestD,-1,1)
maskTestD      = np.moveaxis(maskTestD,-1,1)
dataTestNoNaND = np.moveaxis(dataTestNoNaND,-1,1)
            
# set to NaN patch boundaries
#dataTraining[:,0:5,:] =  dataTrainingNoNaN[:,0:5,:]
#dataTest[:,0:5,:]     =  dataTestNoNaN[:,0:5,:]
    
############################################
## raw data
X_trainD         = dataTrainingNoNaND
        
X_train_missingD = dataTrainingD
mask_trainD      = maskTrainingD
        
X_valD         = dataValNoNaND
X_val_missingD = dataValD
mask_valD      = maskValD
        
X_testD         = dataTestNoNaND
X_test_missingD = dataTestD
mask_testD      = maskTestD
            
############################################
## normalized data wrt to each measurement station
        
if flagTypeMissData ==2 :
    mean2 = np.mean(X_train_missingD[:],0)
    mean2mask = np.mean(mask_trainD[:],0)


    mean3 = np.mean(mean2,1)
    mean3 = mean3.reshape(31,1)
    print(mean3)
    mean3mask = np.mean(mean2mask,1)
    mean3mask = mean3mask.reshape(31,1)
    print(mean3mask)
    meanTr          = mean3/mean3mask
    print(meanTr)
    mean2true = np.mean(X_trainD[:],0)
    mean3true = np.mean(mean2true,1)
    mean3true = mean3true.reshape(31,1)
    meanTrtrue = mean3true
    print(meanTrtrue)
            
    meansquaretrue = np.mean( (X_trainD-meanTrtrue)**2,0)
    meansquare2true = np.mean(meansquaretrue,1)
    meansquare2true=meansquare2true.reshape(31,1)
    stdTrtrue           = np.sqrt(meansquare2true )
    print(stdTrtrue)
            
            
    x_train_missingD = X_train_missingD - meanTr*mask_trainD
            
    x_val_missingD = X_val_missingD - meanTr*mask_valD
    x_test_missingD  = X_test_missingD - meanTr*mask_testD
            
    # scale wrt to each station
    meansquare = np.mean( X_train_missingD**2,0)
    meansquare2 = np.mean(meansquare,1)
    meansquare2=meansquare2.reshape(31,1)
    stdTr           = np.sqrt(meansquare2 / mean3mask)
            
    x_train_missingD = x_train_missingD / stdTr
    x_val_missingD = x_val_missingD / stdTr
    x_test_missingD  = x_test_missingD / stdTr
            
    x_trainD = (X_trainD - meanTr) / stdTr
    x_valD = (X_valD - meanTr) / stdTr
    x_testD  = (X_testD - meanTr) / stdTr
            
    print(np.mean(x_train_missingD))
    print(np.mean(x_trainD))
    print(np.mean(x_val_missingD))
    print(np.mean(x_valD))
    print(np.mean(x_test_missingD))
    print(np.mean(x_testD))
            
            
               
elif flagTypeMissData==3 :
    mean2 = np.mean(X_train_missingD[:],0)
    mean3 = np.mean(mean2,1)
    mean3 = mean3.reshape(31,1)
    meanTr = mean3
    x_train_missingD = X_train_missingD - meanTr
    x_val_missingD = X_val_missingD - meanTr
    x_test_missingD  = X_test_missingD - meanTr
    meansquare = np.mean( x_train_missingD**2,0)
    meansquare2 = np.mean(meansquare,1)
    meansquare2=meansquare2.reshape(31,1)
            
    stdTr           = np.sqrt(meansquare2)
            
    for i in MaskedStations :
        stdTr[i-1] =1
    print(stdTr)
    print(X_trainD[0,:,:])
    x_train_missingD = x_train_missingD / stdTr
    x_val_missingD = x_val_missingD / stdTr
    x_test_missingD  = x_test_missingD / stdTr
    mean2true = np.mean(X_trainD[:],0)
    mean3true = np.mean(mean2true,1)
    mean3true = mean3true.reshape(31,1)
    meanTrtrue = mean3true
            
    meansquaretrue = np.mean( (X_trainD-meanTrtrue)**2,0)
    meansquare2true = np.mean(meansquaretrue,1)
    meansquare2true=meansquare2true.reshape(31,1)
    stdTrtrue           = np.sqrt(meansquare2true )
    print(stdTrtrue)
    print(meanTrtrue)
            
    x_trainD = (X_trainD - meanTrtrue) / stdTrtrue
    x_valD = (X_valD - meanTrtrue) / stdTrtrue
    x_testD  = (X_testD - meanTrtrue) / stdTrtrue
            
elif flagTypeMissData == 4 :
    #Moyenne des débits par station sur l'échatillon total
    M = np.mean(X_trainD,0)
    mean_X_trainD = np.mean(M,1)
    mean_X_trainD = mean_X_trainD.reshape(31,1)
    meanTr        = mean_X_trainD
    meanTrtrue = meanTr
    #Moyenne des débits
    X_nomask=X_trainD[:,:,:(int(2*DT/3+1))]
    M2 = np.mean(X_nomask,0)
    mean_X_train_missingD = np.mean(M2,1)
    mean_X_train_missingD = mean_X_train_missingD.reshape(31,1)

    #Ecart-type:
    meansquaretrue = np.mean( (X_trainD-mean_X_trainD)**2,0)
    meansquare2true = np.mean(meansquaretrue,1)
    meansquare2true=meansquare2true.reshape(31,1)
    stdTrtrue           = np.sqrt(meansquare2true )
    
    #Normalisation et standardisation des données
    x_train_missingD = np.zeros(X_train_missingD.shape)
    x_train_missingD[:,:,:(int(2*DT/3)+1)]=(X_train_missingD[:,:,:(int(2*DT/3)+1)]-mean_X_trainD)/stdTrtrue
 
    x_val_missingD = np.zeros(X_val_missingD.shape)
    x_val_missingD[:,:,:(int(2*DT/3)+1)]=(X_val_missingD[:,:,:(int(2*DT/3)+1)]-mean_X_trainD)/stdTrtrue

    x_test_missingD = np.zeros(X_test_missingD.shape)
    x_test_missingD[:,:,:(int(2*DT/3)+1)]=(X_test_missingD[:,:,:(int(2*DT/3)+1)]-mean_X_trainD)/stdTrtrue


    x_trainD = (X_trainD - mean_X_trainD) / stdTrtrue
    x_valD = (X_valD - mean_X_trainD) / stdTrtrue
    x_testD  = (X_testD - mean_X_trainD) / stdTrtrue
    
    stdTr =stdTrtrue
    
else : 
    mean2 = np.mean(X_train_missingD[:],0)
    mean2mask = np.mean(mask_trainD[:],0)
            

    mean3 = np.mean(mean2,1)
    mean3 = mean3.reshape(31,1)
    print(mean3)
    mean3mask = np.mean(mean2mask,1)
    mean3mask = mean3mask.reshape(31,1)
    print(mean3mask)
    meanTr          = mean3/mean3mask
    print(meanTr)
            
    x_train_missingD = X_train_missingD - meanTr*mask_trainD
    x_val_missingD = X_val_missingD - meanTr
    x_test_missingD  = X_test_missingD - meanTr
            
    # scale wrt to each station
    meansquare = np.mean( X_train_missingD**2,0)
    meansquare2 = np.mean(meansquare,1)
    meansquare2=meansquare2.reshape(31,1)
    stdTr           = np.sqrt(meansquare2 / mean3mask)
            
    x_train_missingD = x_train_missingD / stdTr
    x_val_missingD = x_val_missingD / stdTr
    x_test_missingD  = x_test_missingD / stdTr
            
    x_trainD = (X_trainD - meanTr) / stdTr
    x_valD = (X_valD - meanTr) / stdTr
    x_testD  = (X_testD - meanTr) / stdTr
            
# Generate noisy observsation
        
#X_train_obsD = X_train_missingD + sigNoise * maskTrainingD * np.random.randn(X_train_missingD.shape[0],X_train_missingD.shape[1],X_train_missingD.shape[2])
#X_val_obsD = X_val_missingD + sigNoise * maskValD * np.random.randn(X_val_missingD.shape[0],X_val_missingD.shape[1],X_val_missingD.shape[2])
#X_test_obsD  = X_test_missingD  + sigNoise * maskTestD * np.random.randn(X_test_missingD.shape[0],X_test_missingD.shape[1],X_test_missingD.shape[2])
            
#x_train_obsD = (X_train_obsD - meanTr) / stdTr
#x_val_obsD = (X_val_obsD - meanTr) / stdTr
#x_test_obsD  = (X_test_obsD - meanTr) / stdTr
        
#Without noise :
X_train_obsD = X_train_missingD 
X_val_obsD = X_val_missingD 
X_test_obsD  = X_test_missingD
        
x_train_obsD = x_train_missingD
x_val_obsD = x_val_missingD
x_test_obsD = x_test_missingD 

m_seuil = ((seuil_10.reshape(31,1)-meanTr)/stdTr)
print(seuil_10.shape)
print(meanTr.shape)
print(stdTr.shape)
print("seuil normalisé")
print(m_seuil)
        
print('..... Training dataset: %dx%dx%d'%(x_train_missingD.shape[0],x_trainD.shape[1],x_trainD.shape[2]))
print('..... Validation dataset: %dx%dx%d'%(x_valD.shape[0],x_valD.shape[1],x_valD.shape[2]))
print('..... Test dataset    : %dx%dx%d'%(x_testD.shape[0],x_testD.shape[1],x_testD.shape[2]))
            

print('........ Initialize interpolated states')
## Initial interpolation
#flagInit = 0 : Masked values are replaced by 0
#flagInit = 1 : masked values are replaced by last available value (prevision)
#flaginit = 2 : Interpolation 

flagInit = 1

            
if flagInit == 0: 
    X_train_InitD = mask_trainD * X_train_obsD + (1. - mask_trainD) * (np.zeros(X_train_missingD.shape) + meanTr)
    X_val_InitD = mask_valD * X_val_obsD + (1. - mask_valD) * (np.zeros(X_val_missingD.shape) + meanTr)
    X_test_InitD  = mask_testD * X_test_obsD + (1. - mask_testD) * (np.zeros(X_test_missingD.shape) + meanTr)
    
elif flagInit==1 :
    X_ext_train = X_train_missingD[:,:,int(2*DT/3)].reshape(X_train_missingD.shape[0],X_train_missingD.shape[1],1)
    X_train_InitD = mask_trainD * X_train_obsD + (1. - mask_trainD)*X_ext_train
    X_ext_val = X_val_missingD[:,:,int(2*DT/3)].reshape(X_val_missingD.shape[0],X_val_missingD.shape[1],1)
    X_val_InitD = mask_valD * X_val_obsD + (1. - mask_valD)*X_ext_val
    X_ext_test = X_test_missingD[:,:,int(2*DT/3)].reshape(X_test_missingD.shape[0],X_test_missingD.shape[1],1)
    X_test_InitD = mask_testD * X_test_obsD + (1. - mask_testD)*X_ext_test
    
    
    
    
else:
    X_train_InitD = np.zeros(X_trainD.shape)
    for ii in range(0,X_trainD.shape[0]):
        # Initial linear interpolation for each component
        XInitD = np.zeros((X_trainD.shape[1],X_trainD.shape[2]))
           
        for kk in range(0,mask_trainD.shape[1]):
            indt  = np.where( mask_trainD[ii,kk,:] == 1.0 )[0]
            indt_ = np.where( mask_trainD[ii,kk,:] == 0.0 )[0]
           
            if len(indt) > 1:
                indt_[ np.where( indt_ < np.min(indt)) ] = np.min(indt)
                indt_[ np.where( indt_ > np.max(indt)) ] = np.max(indt)
                fkk = scipy.interpolate.interp1d(indt, X_train_obsD[ii,kk,indt])
                XInitD[kk,indt]  = X_train_obsD[ii,kk,indt]
                XInitD[kk,indt_] = fkk(indt_)
            else:
                XInitD = XInitD + meanTr
            
        X_train_InitD[ii,:,:] = XInitD
            
    X_val_InitD = np.zeros(X_valD.shape)
    for ii in range(0,X_valD.shape[0]):
        # Initial linear interpolation for each component
        XInitD = np.zeros((X_valD.shape[1],X_valD.shape[2]))
           
        for kk in range(0,mask_valD.shape[1]):
            indt  = np.where( mask_valD[ii,kk,:] == 1.0 )[0]
            indt_ = np.where( mask_valD[ii,kk,:] == 0.0 )[0]
           
            if len(indt) > 1:
                indt_[ np.where( indt_ < np.min(indt)) ] = np.min(indt)
                indt_[ np.where( indt_ > np.max(indt)) ] = np.max(indt)
                fkk = scipy.interpolate.interp1d(indt, X_val_obsD[ii,kk,indt])
                XInitD[kk,indt]  = X_val_obsD[ii,kk,indt]
                XInitD[kk,indt_] = fkk(indt_)
            else:
                XInitD = XInitD + meanTr
            
        X_val_InitD[ii,:,:] = XInitD
            
    X_test_InitD = np.zeros(X_testD.shape)
    for ii in range(0,X_testD.shape[0]):
        # Initial linear interpolation for each component
        XInit = np.zeros((X_testD.shape[1],X_testD.shape[2]))
            
        for kk in range(0,X_testD.shape[1]):
            indt  = np.where( mask_testD[ii,kk,:] == 1.0 )[0]
            indt_ = np.where( mask_testD[ii,kk,:] == 0.0 )[0]
            
            if len(indt) > 1:
                indt_[ np.where( indt_ < np.min(indt)) ] = np.min(indt)
                indt_[ np.where( indt_ > np.max(indt)) ] = np.max(indt)
                fkk = scipy.interpolate.interp1d(indt, X_test_obsD[ii,kk,indt])
                XInit[kk,indt]  = X_test_obsD[ii,kk,indt]
                XInit[kk,indt_] = fkk(indt_)
            else:
                XInit = XInit + meanTr
        
        X_test_InitD[ii,:,:] = XInit
        #plt.figure()
        #plt.figure()
        #plt.plot(YObs[0:200,1],'r.')
        #plt.plot(XGT[0:200,1],'b-')
        #plt.plot(XInit[0:200,1],'k-')
                        
x_train_InitD = ( X_train_InitD - meanTr ) / stdTr
x_val_InitD = ( X_val_InitD - meanTr ) / stdTr
x_test_InitD = ( X_test_InitD - meanTr ) / stdTr
        
# reshape to dT-1 for time dimension
DT = DT-1
X_train_obsD        = X_train_obsD[:,:,0:DT]
X_trainD            = X_trainD[:,:,0:DT]
X_train_missingD    = X_train_missingD[:,:,0:DT]
mask_trainD         = mask_trainD[:,:,0:DT]
            
x_train_obsD        = x_train_obsD[:,:,0:DT]
x_trainD            = x_trainD[:,:,0:DT]
x_train_InitD       = x_train_InitD[:,:,0:DT]
X_train_InitD       = X_train_InitD[:,:,0:DT]
        
X_val_obsD        = X_val_obsD[:,:,0:DT]
X_valD            = X_valD[:,:,0:DT]
X_val_missingD    = X_val_missingD[:,:,0:DT]
mask_valD         = mask_valD[:,:,0:DT]
            
x_val_obsD        = x_val_obsD[:,:,0:DT]
x_valD            = x_valD[:,:,0:DT]
x_val_InitD       = x_val_InitD[:,:,0:DT]
X_val_InitD       = X_val_InitD[:,:,0:DT]

X_test_obsD        = X_test_obsD[:,:,0:DT]
X_testD            = X_testD[:,:,0:DT]
X_test_missingD    = X_test_missingD[:,:,0:DT]
mask_testD         = mask_testD[:,:,0:DT]

x_test_obsD        = x_test_obsD[:,:,0:DT]
x_testD            = x_testD[:,:,0:DT]
x_test_InitD       = x_test_InitD[:,:,0:DT]
X_test_InitD       = X_test_InitD[:,:,0:DT]

print('..... Training dataset: %dx%dx%d'%(x_trainD.shape[0],x_trainD.shape[1],x_trainD.shape[2]))
print('..... Validation dataset: %dx%dx%d'%(x_valD.shape[0],x_valD.shape[1],x_valD.shape[2]))
print('..... Test dataset    : %dx%dx%d'%(x_testD.shape[0],x_testD.shape[1],x_testD.shape[2]))

........ Data extraction
........ Random seed set to 100
[2220.  1020.   764.   744.   577.   535.   511.   329.   279.   218.
   92.3   45.  1240.   257.   250.   116.    81.5   46.7   35.3  179.
  130.   123.    96.5   78.6   69.    62.1   47.6   81.8   74.4  449.
  427. ]
(874, 13, 31)
(874, 13, 31)
... Data type: _ObsSubRnd_50_20
(31,)
(31, 1)
(31, 1)
seuil normalisé
[[1.31824764]
 [1.26866338]
 [1.25838101]
 [1.25653513]
 [1.21610044]
 [1.20150933]
 [1.2163678 ]
 [1.19586817]
 [1.22944378]
 [1.24693638]
 [1.17121264]
 [1.26490812]
 [1.30504977]
 [1.24744332]
 [1.32544306]
 [1.09160007]
 [1.1116852 ]
 [0.87639607]
 [0.76977733]
 [1.07203351]
 [1.04262843]
 [1.14722608]
 [1.15214541]
 [1.14882606]
 [1.05608624]
 [1.03135639]
 [1.07488227]
 [1.21720527]
 [1.19016786]
 [1.21929718]
 [1.25150911]]
..... Training dataset: 13478x31x13
..... Validation dataset: 874x31x13
..... Test dataset    : 874x31x13
........ Initialize interpolated states
..... Training dataset: 13478x31x12
..... Val

In [12]:
print(var_Tr)
print(var_Tt)
print(var_Val)
 
print(mean_Tt)
print(mean_Val)


1.000430744763107
1.047662421960815
1.0661778697596147
0.12310378896799264
0.13217613565121983


..... Training dataset: 13110x31x20
..... Validation dataset: 506x31x20
..... Test dataset    : 506x31x20


In [61]:
class LitModel(pl.LightningModule):
    def __init__(self, hparam, *args, **kwargs):
        super().__init__()
        h= hparam if isinstance(hparam, dict) else OmegaConf.to_container(hparam, resolve=True)
        self.save_hyperparameters(h)
        
        print(self.hparams)
        #self.save_hyperparameters(hparams)
        self.var_Val = kwargs['var_Val']
        self.var_Tr = kwargs['var_Tr']
        self.var_Tt = kwargs['var_Tt']

        # create longitudes & latitudes coordinates
        
        print(self.hparams.dT)
        self.shapeData = [self.hparams.dT*2,self.hparams.NbStations]

In [62]:
mod = LitModel(hparam = cfg, w_loss= wLoss,
                               mean_Tr=meanTrtrue, mean_Tt=mean_Tt, mean_Val=mean_Val,var_Tr=var_Tr,var_Tt=var_Tt,var_Val=var_Val)
                             #var_Tr, var_Tt, var_Val,
                               
       

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()